In [2]:
import cobra 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols
from sympy.parsing.sympy_parser import parse_expr

# Load the model
model = cobra.io.read_sbml_model("C:/Users/iamab/iGEM-IITB/FBA/rFBA/models/ecoli_core.xml")

#initalise the bounds here!!!!





# Load other data

#read from json as dictionary
import json
genes_to_rule = {}        ##################################################
with open('C:/Users/iamab/iGEM-IITB/FBA/rFBA/models/core_regulatory_rules.json', 'r') as fp:
    genes_to_rule = json.load(fp)

for key, value in genes_to_rule.items():

    #converts values strings to exprs
    genes_to_rule[key] = parse_expr(value.replace("NOT", "~").replace("AND", "&").replace("OR", "|").replace("[", "_").replace("]", "").replace("-", "_"))

#read from csv as list
met_df = pd.read_csv('C:/Users/iamab/iGEM-IITB/FBA/rFBA/models/met_list.csv')
rxn_df = pd.read_csv('C:/Users/iamab/iGEM-IITB/FBA/rFBA/models/rxn_list.csv')

rxn_list = rxn_df['reaction inputs'].tolist() ##################################################

met_list = met_df['metabolite inputs'].tolist() ##################################################

symbol_translation_backwards = {}  ##################################################
symbol_translation_forwards = {}   ##################################################

for i in range(len(met_df['translated'])):
    symbol_translation_backwards[met_df["translated"][i]] = met_df["metabolite inputs"][i]
    symbol_translation_forwards[met_df["metabolite inputs"][i]] = met_df["translated"][i]

genes_to_var = {}
for x in genes_to_rule.keys():
    genes_to_var[x] = 1

#initiliazing genes to vars
genes_to_var['aceA'] = 0
genes_to_var['aceB'] = 1
genes_to_var['aceE'] = 0
#....

#making a dictionary of metabolites to inital concentrations
initial_concs_met = {}
for met in met_list:
    initial_concs_met[met] = 0

#open bnum to gene and gene to bnum
gene_to_bnum = {}
bnum_to_gene = {}
with open('C:/Users/iamab/iGEM-IITB/FBA/rFBA/models/gene_to_bnum.json', 'r') as fp:
    gene_to_bnum = json.load(fp)
with open('C:/Users/iamab/iGEM-IITB/FBA/rFBA/models/bnum_to_gene.json', 'r') as fp:
    bnum_to_gene = json.load(fp)



#making a dictionary of reactions in rxnlist to irules
rxns_to_rule = {}
for rxn in rxn_list:
    rxns_to_rule[rxn] = model.reactions.get_by_id(rxn).gene_reaction_rule
    expr = parse_expr(rxns.replace("not", "~").replace("and", "&").replace("or", "|").replace("[", "_").replace("]", "").replace("-", "_"))


    
#making a dictionary of reactions to vars and initial bounds
rxns_to_var_and_bounds = {}
for rxn in rxn_list:
    #get the reaction from the model
    rxn_obj = model.reactions.get_by_id(rxn)
    #get the bounds
    lb = rxn_obj.lower_bound
    ub = rxn_obj.upper_bound
    if lb == 0 and ub == 0:
        rxns_to_var_and_bounds[rxn] = [0, 0, 0]
    else:
        rxns_to_var_and_bounds[rxn] = [1, lb, ub]


# class state:
#     def __init__(self, model, initial_substrate_concs, supply, initial_X, tim,objective):
#         self.model = model
#         self.supply = supply #dictionary from uptake metabolites to supply values
#         self.substrate_concs = initial_substrate_concs #dictionary from metabolite to a list of concentrations
#         self.X = []
#         self.X.append(initial_X)
#         self.del_T = tim # in mins
#         self.counter = 0
#         if objective is None:
#             model.objective = 'Biomass_Ecoli_core_w_GAM'
#         else:
#             model.objective = objective
#         self.genes_to_rules = 
            
        
#     def convert_S2f(self,S_c,X,del_T):
#         #S_c is the concentration of the substrate
#         #X is the biomass concentration
#         #del_T is the time interval
#         #returns the flux of the substrate
#         return (S_c/(X*del_T))
        
#     def update_model(self):
#         print("_"*50)
#         print("Time: ",self.counter*self.del_T)
#         print("_"*50)
#         for i, (rxn_id, conc) in enumerate(self.substrate_concs.items()):
#             S_c = conc[self.counter]
#             flux = self.convert_S2f(S_c,self.X[self.counter],self.del_T)
#             if model.reactions.get_by_id(rxn_id).upper_bound < -flux:
#                 model.reactions.get_by_id(rxn_id).upper_bound = 0
#             else:
#                 model.reactions.get_by_id(rxn_id).lower_bound = -flux
#                 model.reactions.get_by_id(rxn_id).upper_bound = 0
#         soln = model.optimize()
#         #extracting the values now
#         mu = soln.objective_value
#         d1 = {}
#         for i, (rxn_id, conc) in enumerate(self.substrate_concs.items()):
#             d1[rxn_id] = soln.fluxes[rxn_id]
        
#         new_X = self.X[self.counter]*(np.exp(mu*(self.del_T/60)))
#         for i, (rxn_id, conc) in enumerate(self.substrate_concs.items()):
#             self.substrate_concs[rxn_id].append(self.substrate_concs[rxn_id][self.counter] + (-d1[rxn_id]/mu)*(self.X[self.counter])*(1-np.exp(mu*(self.del_T/60))))
#         self.X.append(new_X)      
#         self.counter+=1
        
    
# # Define the initial conditions for the substrates and biomass
# glc_rx = 'EX_glc_e_'
# initial_substrate_concs = {glc_rx: [10]}  # example concentrations
# initial_biomass = 0.03  # example initial biomass
# timesteps = 300
# # print(model.objective)
# # soln1 = model.optimize()
# # print(soln1)
# model_state = state(model, initial_substrate_concs, None, initial_biomass, 1, "Biomass_Ecoli_core_N__w_GAM_")
# for i in range(timesteps):
#     model_state.update_model()
    
# X = np.arange(0,timesteps+1,1)
# Y = model_state.substrate_concs[glc_rx]
# print(Y)
# # plt.plot(X,Y)
# Y1 = model_state.X
# # plt.plot(X,Y1)
# plt.plot(Y1,Y)
# plt.show()


Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_13dpg_c "M_3_Phospho_D_glyceroyl_phosphate_C3H4O10P2">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_2pg_c "M_D_Glycerate_2_phosphate_C3H4O7P">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_3pg_c "M_3_Phospho_D_glycerate_C3H4O7P">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_6pgc_c "M_6_Phospho_D_gluconate_C6H10O10P">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_6pgl_c "M_6_phospho_D_glucono_1_5_lactone_C6H9O9P">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_ac_c "M_Acetate_C2H3O2">
Use of the specie

Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_etoh_b "M_Ethanol_C2H6O">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_for_b "M_Formate_CH1O2">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_fum_b "M_Fumarate_C4H2O4">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_glc_D_b "M_D_Glucose_C6H12O6">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_h2o_b "M_H2O_H2O">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_h_b "M_H_H">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_lac_D_b "M_D_Lactate_C3H5O3">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_o2_b "M_O2_O2">
Use of the species charge attribute is discouraged, use fbc:charge instead: <Species M_pi_b "M_Phosphate_HO4P">
Use o

Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_EX_succ_e_ "R_Succinate_exchange">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_FBA "R_fructose_bisphosphate_aldolase">
Use of GENE ASSOCIATION or GENE_ASSOCIATION in the notes element is discouraged, use fbc:gpr instead: <Reaction R_FBA "R_fructose_bisphosphate_aldolase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_FBP "R_fructose_bisphosphatase">
Use of GENE ASSOCIATION or GENE_ASSOCIATION in the notes element is discouraged, use fbc:gpr instead: <Reaction R_FBP "R_fructose_bisphosphatase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_FORt "R_formate_transport_via_diffusion">
Use of GENE ASSOCIATION or GENE_ASSOCIATION in the notes element is discouraged, use fbc:gpr instead: <Reaction R_FORt "R

Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_PPC "R_phosphoenolpyruvate_carboxylase">
Use of GENE ASSOCIATION or GENE_ASSOCIATION in the notes element is discouraged, use fbc:gpr instead: <Reaction R_PPC "R_phosphoenolpyruvate_carboxylase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_PPCK "R_phosphoenolpyruvate_carboxykinase">
Use of GENE ASSOCIATION or GENE_ASSOCIATION in the notes element is discouraged, use fbc:gpr instead: <Reaction R_PPCK "R_phosphoenolpyruvate_carboxykinase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds instead: <Reaction R_PPS "R_phosphoenolpyruvate_synthase">
Use of GENE ASSOCIATION or GENE_ASSOCIATION in the notes element is discouraged, use fbc:gpr instead: <Reaction R_PPS "R_phosphoenolpyruvate_synthase">
Encoding LOWER_BOUND and UPPER_BOUND in KineticLaw is discouraged, use fbc:fluxBounds inste

Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_EX_fum_e_ "R_Fumarate_exchange">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_EX_glc_e_ "R_D_Glucose_exchange">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_EX_h2o_e_ "R_H2O_exchange">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_EX_h_e_ "R_H_exchange">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_EX_lac_D_e_ "R_D_lactate_exchange">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_EX_o2_e_ "R_O2_exchange">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discouraged, use fbc:fluxObjective instead: <Reaction R_EX_pi_e_ "R_Phosphate_exchange">
Encoding OBJECTIVE_COEFFICIENT in KineticLaw is discourage

KeyError: 'translated'